In [27]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    
    from elasticsearch import helpers
    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Loaded  .. . . . . . . .


In [28]:
df = pd.read_excel("elastic_Search_Data.xlsx")
print("read")

read


In [29]:
df.head()

,filename,country,Product name,Refined Product Name,para_text,Possible Questions,Comment
0,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is indicated f...,What is ATACAND used for\nWhat is ATACAND used...,NaN
1,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Geriatrics (> 65 years of age) : No overall di...,What is the efficacy of ATACAND in elderly pop...,NaN
2,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),Pediatrics (6 to 17 years of age): \n•\tHypert...,What is ATACAND used for in children\nWhat is ...,NaN
3,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),ATACAND (candesartan cilexetil) is contraindic...,Who is ATACAND conracindicated in\nWho is ATAC...,NaN
4,Candesartan_PM_Reviewed.docx,Canada,Candesartan cilexetil,Atacand (Candesartan Cilexetil),There is evidence that co-administration of an...,Can ATACAND be co-administered with aliskiren\...,NaN


In [30]:
from bert_serving.client import BertClient
bc = BertClient()

In [31]:
answers=list(df["para_text"])

In [32]:
answers

['ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'Geriatrics (> 65 years of age) : No overall differences in safety or effectiveness were observed between these subjects and younger subjects, and other reported clinical experience has not identified differences in responses between the elderly and younger patients, but greater sensitivity of some older individuals cannot be ruled out. ',
 'Pediatrics (6 to 17 years of age): \n•\tHypertension\nATACAND is indicated for the treatment of essential hypertension in children and adolescents 6 to 17 years of age (se

In [33]:
ml_vec=bc.encode(answers)

In [34]:
ml_vec

array([[-0.21258354, -0.09253826, -0.0774114 , ...,  0.18896478,
        -0.20395526,  0.00088365],
       [-0.17276916, -0.3651918 , -0.11895184, ..., -0.12006989,
         0.10894839,  0.0040433 ],
       [-0.27216503, -0.07298217, -0.04816339, ...,  0.12562552,
         0.15384367,  0.1262906 ],
       ...,
       [-0.09729752,  0.15539923,  0.02649379, ..., -0.07838615,
         0.0778973 ,  0.42761424],
       [-0.08624224, -0.06090773,  0.04918787, ..., -0.00910198,
         0.27791575, -0.0777555 ],
       [ 0.00739385,  0.07879531,  0.02765172, ...,  0.05927146,
        -0.04922816,  0.01016224]], dtype=float32)

In [35]:
type(ml_vec[0])

numpy.ndarray

In [36]:
ml_vec[0]

array([-2.12583542e-01, -9.25382599e-02, -7.74113983e-02,  2.08734229e-01,
        2.33869433e-01, -1.54881388e-01, -1.55369997e-01, -2.10733041e-01,
        2.48279329e-02, -1.23438388e-01,  2.50698477e-01, -4.60333424e-03,
        2.35185642e-02, -2.72644192e-01,  3.34664136e-01, -4.34827507e-02,
       -2.80491579e-02, -3.24270159e-01, -2.32795045e-01,  2.65478224e-01,
        2.06407875e-01, -5.23197055e-02,  1.10645862e-02, -2.33840644e-02,
       -4.47062887e-02, -1.69695988e-01,  2.29911894e-01,  4.34144795e-01,
       -1.35025412e-01,  3.69068325e-01, -2.88098883e-02, -3.92821878e-02,
       -5.19309156e-02, -4.03696336e-02,  2.58596748e-01,  1.59972101e-01,
        2.76813726e-03, -8.09476152e-02,  1.10780962e-01, -5.16055971e-02,
        1.11025952e-01, -4.76526516e-03, -3.72431189e-01, -3.65642339e-01,
       -2.21345603e-01,  5.39731272e-02, -3.39946263e-02,  1.13263033e-01,
       -1.13086320e-01, -1.79629683e-01,  3.02737147e-01,  1.84787050e-01,
       -1.10104777e-01, -

In [37]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "ml_vector":{
         "type":"dense_vector",
         "dims":768
      } 
    }
   }
}

In [38]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

python client created


In [39]:
#creating the database
IndexName = 'thisisit'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [40]:
my

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [thisisit/-uRgIye5QB6mv0IsLzRmsw] already exists',
    'index_uuid': '-uRgIye5QB6mv0IsLzRmsw',
    'index': 'thisisit'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [thisisit/-uRgIye5QB6mv0IsLzRmsw] already exists',
  'index_uuid': '-uRgIye5QB6mv0IsLzRmsw',
  'index': 'thisisit'},
 'status': 400}

In [41]:
df.columns

Index(['filename', 'country', 'Product name', 'Refined Product Name',
       'para_text', 'Possible Questions', 'Comment'],
      dtype='object')

In [42]:
df22 = df.to_dict('records')
df22

[{'filename': 'Candesartan_PM_Reviewed.docx',
  'country': 'Canada',
  'Product name': 'Candesartan cilexetil',
  'Refined Product Name': 'Atacand (Candesartan Cilexetil)',
  'para_text': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'Possible Questions': 'What is ATACAND used for\nWhat is ATACAND used for in Canada\nHow can ATACAND be used\nHow can ATACAND be used in Canada\nCan ATACAND be used with calcium channel blockers\nCan ATACAND be used with calcium channel blockers in Canada\nCan ATACAND (candesartan cilexetil) be indicated for heartfailure',
  

In [43]:
len(df22)

99

In [44]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("para_text",""),
    'ml_vector':ml_vec[c]})

In [45]:
arr

[{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
  'ml_vector': array([-2.12583542e-01, -9.25382599e-02, -7.74113983e-02,  2.08734229e-01,
          2.33869433e-01, -1.54881388e-01, -1.55369997e-01, -2.10733041e-01,
          2.48279329e-02, -1.23438388e-01,  2.50698477e-01, -4.60333424e-03,
          2.35185642e-02, -2.72644192e-01,  3.34664136e-01, -4.34827507e-02,
         -2.80491579e-02, -3.24270159e-01, -2.32795045e-01,  2.65478224e-01,
          2.06407875e-01, -5.23197055e-02,  1.10645862e-02, -2.33840644e-02,
         -4.47062887e-02, -1.6

In [46]:
arr[0]

{'answer': 'ATACAND (candesartan cilexetil) is indicated for: \n\n•\tHypertension\no\tThe treatment of mild to moderate essential hypertension.\no\tATACAND may be used alone or concomitantly with thiazide diuretics.\no\tThe safety and efficacy of concurrent use with calcium channel blockers have not been established.\n•\tHeart Failure\no\tThe treatment of NYHA Class II and III heart failure with ejection fraction\n≤ 40% in addition to standard therapy, with or without an ACE inhibitor. ',
 'ml_vector': array([-2.12583542e-01, -9.25382599e-02, -7.74113983e-02,  2.08734229e-01,
         2.33869433e-01, -1.54881388e-01, -1.55369997e-01, -2.10733041e-01,
         2.48279329e-02, -1.23438388e-01,  2.50698477e-01, -4.60333424e-03,
         2.35185642e-02, -2.72644192e-01,  3.34664136e-01, -4.34827507e-02,
        -2.80491579e-02, -3.24270159e-01, -2.32795045e-01,  2.65478224e-01,
         2.06407875e-01, -5.23197055e-02,  1.10645862e-02, -2.33840644e-02,
        -4.47062887e-02, -1.69695988e

In [47]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [48]:
resp

{'_index': 'thisisit',
 '_type': '_doc',
 '_id': '98',
 '_version': 7,
 'result': 'updated',
 '_shards': {'total': 1, 'successful': 1, 'failed': 0},
 '_seq_no': 692,
 '_primary_term': 6}

In [61]:
#testing
que = "How does ATACAND affect adults who have undergone in utero exposure "
que_vec=bc.encode([que])

In [62]:
que_vec

array([[-3.64449322e-02, -2.53909566e-02, -8.58423635e-02,
        -6.81598559e-02, -1.17318988e-01, -4.99909490e-01,
         2.46220961e-01, -7.23915324e-02,  3.09959054e-01,
        -5.52141130e-01,  3.76222789e-01, -1.43828124e-01,
         2.80368090e-01, -4.00146216e-01,  1.08074613e-01,
         2.03542084e-01,  5.88609539e-02, -2.69451626e-02,
        -2.52113678e-02,  2.05153570e-01,  2.50235181e-02,
        -1.23245269e-01,  1.36484683e-01, -1.68881938e-01,
         4.48713511e-01,  5.98021038e-02, -2.45961413e-01,
         5.86933613e-01,  3.10363263e-01, -2.85527140e-01,
         6.00893833e-02, -1.50078371e-01,  1.95046961e-02,
        -4.72535461e-01,  1.62101984e-02,  1.17220603e-01,
         5.11469245e-01,  1.13726236e-01, -9.74947810e-02,
        -2.11700886e-01,  1.29370719e-01, -1.16756037e-02,
        -3.31609339e-01, -1.99078798e-01, -1.51799187e-01,
         3.98691118e-01,  3.07458714e-02,  9.41378530e-04,
         1.66112959e-01, -5.41766956e-02,  3.39434206e-0

In [63]:

script_query = {
"script_score": {
"query": {"match_all":{}},
"script": {
"source": "cosineSimilarity(params.query_vector, doc['ml_vector']) ",
"params": {"query_vector": que_vec[0] }
}}}

In [64]:
es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 99, 'relation': 'eq'},
  'max_score': 0.9325755,
  'hits': [{'_index': 'thisisit',
    '_type': '_doc',
    '_id': '30',
    '_score': 0.9325755,
    '_source': {'answer': 'Type 1 diabetes : There is no experience regarding the administration of ATACAND in children aged 6 to <17 years with type 1 diabetes. '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '28',
    '_score': 0.9315943,
    '_source': {'answer': 'There is no experience regarding the administration of ATACAND in children aged 6 to <17 years with a renal transplant. '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '17',
    '_score': 0.9293156,
    '_source': {'answer': 'There is limited experience regarding the administration of ATACAND in adult patients with renal transplant. '}},
   {'_index': 'thisisit',
    '_type': '_doc',
    '_id': '80',
    '_score': 0.922